In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,biasLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [3]:
## change ##
batch_size = 16
clip = .25

Prepare data

In [4]:
train_gen = SequenceGenSpec(train)
#train_gen = mixupWrapper(SequenceGenSpec(train),4e-2) # does not improve
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [9]:
model = Sequential(Dropout(0.5),
                      ResidualBlockGLU(129),
                      Dropout(0.5),
                      ResidualBlockGLU(258),
                      Dropout(0.5),
                      ResidualBlockGLU(516),
                      Dropout(0.5),
                      ResidualBlockGLU(1032),
                      Dropout(0.5),
                      LambdaLayer(lambda x:x.transpose(1,2)),
                      Linear(2064,1,bias=False),
                      LeakyReLU(0.2,True),
                      LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),
                      #biasLayer(1),
                      Linear(1,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [10]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [11]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [12]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.408487835755715, val_loss:2.4409263134002686
epoch:1, train_loss:2.080362990785104, val_loss:2.3904948234558105
epoch:2, train_loss:2.035799066034647, val_loss:2.3998186588287354
epoch:3, train_loss:2.0096456841207466, val_loss:2.47282075881958
epoch:4, train_loss:1.9748888906951134, val_loss:2.437933921813965
Training completed in 47.249391317367554s


Submission

In [85]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [86]:
test_gen = SequenceGenSpecTest(submission.seg_id.tolist())
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [87]:
submission.iloc[:,1] = predict(model,test_gen)

In [88]:
submission.to_csv('../Submission/sub_spec_2d_cnn_shareWeight.csv',index=False)